In [1]:
import os

os.chdir('../')
%pwd

'/Users/lakshayarora/Desktop/Text-Summary-Generator'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int

In [3]:
from textSummaryGenerator.constants import *
from textSummaryGenerator.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            # lighter checkpoint for easier local/Kaggle training
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [5]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk
import torch

/Users/lakshayarora/Desktop/Text-Summary-Generator/venv311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        if torch.cuda.is_available():
            device = "cuda"
        elif torch.backends.mps.is_available():
            device = "mps"
        else:
            device = "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=50,
            # eval_strategy="no",
            # save_strategy="no",
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
            fp16=torch.cuda.is_available(),
            report_to="none",
            disable_tqdm=False,
        ) 

        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["test"],
            eval_dataset=dataset_samsum_pt["validation"],
        )

        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [7]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2026-02-16 19:23:01,584: INFO: common: yaml file: config/config.yaml loaded successfully]
[2026-02-16 19:23:01,586: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-16 19:23:01,586: INFO: common: created directory at: artifacts]
[2026-02-16 19:23:01,587: INFO: common: created directory at: artifacts/model_trainer]
[2026-02-16 19:23:01,766: INFO: _client: HTTP Request: HEAD https://huggingface.co/t5-small/resolve/main/config.json "HTTP/1.1 200 OK"]
[2026-02-16 19:23:01,889: INFO: _client: HTTP Request: HEAD https://huggingface.co/t5-small/resolve/main/tokenizer_config.json "HTTP/1.1 200 OK"]
[2026-02-16 19:23:02,022: INFO: _client: HTTP Request: GET https://huggingface.co/api/models/t5-small/tree/main/additional_chat_templates?recursive=false&expand=false "HTTP/1.1 307 Temporary Redirect"]
[2026-02-16 19:23:02,130: INFO: _client: HTTP Request: GET https://huggingface.co/api/models/google-t5/t5-small/tree/main/additional_chat_templates?recursive=false&expand=false "HTT

Loading weights: 100%|██████████| 131/131 [00:00<00:00, 1301.63it/s, Materializing param=shared.weight]                                                      


[2026-02-16 19:23:02,944: INFO: _client: HTTP Request: HEAD https://huggingface.co/t5-small/resolve/main/generation_config.json "HTTP/1.1 200 OK"]


/Users/lakshayarora/Desktop/Text-Summary-Generator/venv311/lib/python3.11/site-packages/torch/utils/data/dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  super().__init__(loader)


Step,Training Loss
50,181.471953


Writing model shards: 100%|██████████| 1/1 [00:00<00:00,  2.43it/s]
